In [12]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import Browser

In [13]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/Users/thomasadamson/Downloads/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [ ]:
#Start browser session for mars.nasa.gov/news/
browser = init_browser()

url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [20]:
#Use class 'content_title to retrieve article title'
title = soup.find_all('div', class_= 'content_title')[1].text
title

"NASA's Perseverance Mars Rover Mission Honors Navajo Language"

In [25]:
#Use class 'article_teaser_body to retrieve article tag'
tag = soup.find_all('div', class_='article_teaser_body')[0].text
tag

'Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.'

In [26]:
browser.quit()